# Analyse des Films et Recommandations avec PySpark

Dans ce notebook, nous allons explorer un ensemble de données de films et de notes de spectateurs, puis créer un système de recommandations en utilisant la bibliothèque PySpark. Nous utiliserons diverses fonctionnalités de Spark, comme les transformations de données, les calculs statistiques, ainsi que la réduction de dimension et la standardisation des données.

## Étape 1 : Importation des bibliothèques et configuration de l'environnement
Nous commençons par importer les bibliothèques nécessaires et configurer la session Spark.

In [1]:
import sys
try:
    import distutils
except ModuleNotFoundError:
    import setuptools._distutils as distutils
    sys.modules["distutils"] = distutils

Cette partie du script gère l'importation des bibliothèques nécessaires et traite un problème potentiel de module manquant (distutils). Si le module distutils n'est pas disponible, il essaie d'importer setuptools._distutils et le remplace dans le module système.

## Étape 2 : Initialisation de la session Spark
Nous initialisons une session Spark avec 4 Go de mémoire allouée pour le driver et les exécutants.

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, count
from pyspark.sql.functions import explode, split


spark = SparkSession.builder \
    .appName("MoviesRatings") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "4g") \
    .getOrCreate()

## Étape 3 : Chargement des données
Nous chargeons les fichiers CSV contenant les informations des films et des notes.

In [ ]:
movies = spark.read.csv("ml-latest-small/movies.csv", header=True, inferSchema=True)
ratings = spark.read.csv("ml-latest-small/ratings.csv", header=True, inferSchema=True)
movies.show(10)
ratings.show(10)

Nous chargeons les fichiers CSV contenant les informations des films et des notes. header=True indique que la première ligne des fichiers CSV contient les en-têtes de colonne, et inferSchema=True permet à Spark de deviner automatiquement les types de données des colonnes. Les fonctions show(10) affichent les 10 premières lignes des DataFrames movies et ratings.

## Étape 4 : Exploration des données
Affichage des schémas des deux ensembles de données pour mieux comprendre leur structure. Puis nous examinons le nombre total de film, les notes des films et le nombre d'utilisateurs uniques.

In [ ]:
movies.printSchema()
ratings.printSchema()

In [ ]:
print(f"Nombre total de films : {movies.count()}")
print(f"Nombre total de notes : {ratings.count()}")

In [ ]:
print(f"Nombre d'utilisateurs uniques : {ratings.select('userId').distinct().count()}")
print(f"Nombre de films notés : {ratings.select('movieId').distinct().count()}")

Nombre d'utilisateurs uniques : 610
Nombre de films notés : 9724


Cette étape affiche les schémas des deux ensembles de données pour mieux comprendre leur structure. Ensuite, nous comptons le nombre total de films, le nombre total de notes, le nombre d'utilisateurs uniques et le nombre de films notés.

## Étape 5 : Statistiques descriptives
Affichage de la répartition des notes données par les utilisateurs.

In [ ]:
ratings.describe("rating").show()

In [ ]:
from pyspark.sql.functions import col
ratings.groupBy("rating").count().orderBy(col("rating")).show()

In [ ]:
best_movies = ratings.groupBy("movieId") \
    .agg(avg("rating").alias("avg_rating"), count("rating").alias("count_rating")) \
    .filter("count_rating >= 10") \
    .orderBy(col("avg_rating").desc())
best_movies.show(10)

In [ ]:
avg_ratings_per_movie = ratings.groupBy("movieId").count().agg({"count": "avg"})
avg_ratings_per_movie.show()

In [ ]:
movies_genres = movies.withColumn("genre", explode(split(col("genres"), "\\|")))
movies_genres.groupBy("genre").count().orderBy(col("count").desc()).show()


Dans cette étape, nous affichons la répartition des notes données par les utilisateurs. Nous utilisons groupBy pour regrouper les notes par valeurs uniques et les compter. Ensuite, nous affichons les films les mieux notés en calculant la note moyenne (avg_rating) et le nombre de notes (count_rating) pour chaque film. Nous filtrons pour inclure uniquement les films avec au moins 10 notes. Nous calculons également la moyenne des évaluations par film et explorons les genres de films en décomposant les genres multiples en lignes distinctes (explode et split).

In [ ]:
movies=movies.na.drop()
movies.show()

In [ ]:
ratings=ratings.na.drop()
ratings.show()

In [ ]:
movies_r=movies.join(ratings,on="movieId",how="inner")
movies.show()
movies_r.show()

Nous supprimons les valeurs nulles dans les deux ensembles de données (na.drop()). Ensuite, nous effectuons une jointure interne (inner join) des données de films et des évaluations sur la colonne movieId.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


## Convertion du dataframe Spark des ratings et création de l'histogramme de répartion des notes

In [ ]:

ratings_pd = ratings.select("rating").toPandas()

plt.figure(figsize=(8, 6))
sns.histplot(ratings_pd['rating'], bins=10, kde=True, color='blue')
plt.title("Répartition des notes", fontsize=15)
plt.xlabel("Notes", fontsize=12)
plt.ylabel("Nombre d'occurences", fontsize=12)
plt.show()


## Visualisation des genres les plus populaires

In [ ]:
movies_genres_pd = movies_genres.groupBy("genre").count().orderBy(col("count").desc()).toPandas()

plt.figure(figsize=(10, 6))
sns.barplot(data=movies_genres_pd, x="count", y="genre", palette="viridis")
plt.title("Nombre de films par genre", fontsize=15)
plt.xlabel("Nombre de films", fontsize=12)
plt.ylabel("Genre", fontsize=12)
plt.show()


## Visualisation des 10 films les plus notés

In [ ]:
best_movies_pd = best_movies.join(movies, on="movieId").select("title", "avg_rating").orderBy(col("avg_rating").desc()).limit(10).toPandas()

plt.figure(figsize=(10, 6))
sns.barplot(data=best_movies_pd, x="avg_rating", y="title", palette="coolwarm")
plt.title("Top 10 des films les mieux notés", fontsize=15)
plt.xlabel("Note moyenne", fontsize=12)
plt.ylabel("Titre du film", fontsize=12)
plt.show()


## Etape 6 : Modèle de recommandation avec ALS

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row


Nous utilisons l'algorithme ALS (Alternating Least Squares) pour créer un modèle de recommandation. Nous divisons les données en un ensemble d'entraînement et un ensemble de test, entraînons le modèle, et faisons des prédictions.

In [ ]:
x_train,x_test= movies_r.randomSplit([0.8,0.2])


In [ ]:
x_train,x_test= movies_r.randomSplit([0.8,0.2])
als=ALS(maxIter=10, regParam=0.01,userCol="userId",itemCol="movieId",ratingCol="rating",coldStartStrategy="drop",rank=10)



Division des données : Nous divisons les données en ensembles d'entraînement (x_train) et de test (x_test) avec une répartition de 80% pour l'entraînement et 20% pour le test.

Entraînement du modèle : Nous définissons et entraînons un modèle ALS avec 10 itérations (maxIter=10), un paramètre de régularisation de 0.01 (regParam=0.01), et une stratégie pour gérer les valeurs manquantes (coldStartStrategy="drop"). Le modèle est ensuite ajusté (fit) sur les données d'entraînement.

In [ ]:
model=als.fit(x_train)
pred=model.transform(x_test)
pred.show()

Prédictions : Nous faisons des prédictions sur les données de test et les affichons.

In [28]:
eval=RegressionEvaluator(metricName="rmse",labelCol="rating",predictionCol="prediction")

Évaluation du modèle : Nous évaluons la performance du modèle en utilisant l'erreur quadratique moyenne (RMSE) avec RegressionEvaluator.

Recommandations : Nous générons des recommandations pour tous les utilisateurs et tous les films, et nous les affichons.